In [1]:
import pandas as pd
import numpy as np

In [2]:
df= pd.read_csv('../../../DataSets/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
len(df['review'][0])

1761

In [4]:
df['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train,X_test,y_train,y_test= train_test_split(df['review'],df['sentiment'],test_size=0.2,random_state=99)

In [7]:
X_train.shape

(40000,)

In [8]:
X_train

4057     I had a heck of a good time viewing this pictu...
20382    My wife and I both thought this film a watered...
4386     Stay Alive has a very similar story to some As...
42507    I personally liked this movie and am alarmed a...
11197    Words cannot express how poor this film is.<br...
                               ...                        
42697    Those who love the book Jane Eyre as I do (it'...
36008    a mesmerizing film that certainly keeps your a...
46265    Hey guys I'm actually in this movie! I didn't ...
23587    it was a very well written movie, and the acto...
29313    I mean really, really, REALLY high and this mo...
Name: review, Length: 40000, dtype: object

In [9]:
X_train.iloc[0]

"I had a heck of a good time viewing this picture, and was splendidly surprised at its more erudite features. First off, the film is undeniably cheaply-made with its cardboard sets, limited settings, and creative scientific props. The acting ranges from very poor(the two strippers), barely professional(Herb Evers as the leading man), gothic overstatement(Leslie Daniels as the assistant Kurt)to first-rate with Virginia Leith in the title role as the headless victim alive against her will for the benefit of science and her fiancee's lustful passions. The scripting though is very good and the dialogue is fantastic for a movie of this ilk. Issues abound about what role science and medicine have in our lives and what their boundaries should be. This film is a thinking film in many ways. However, don't be too fooled by its real intent. It is a sleazy story about a man obsessed with his aptitude in medical science who wishes to fuse together his dead girlfriend's head with the perfect body, t

# Preprocessing 

In [10]:
import re

CLEANR = re.compile('<.*?>') 
def strip_html(data):
    cleantext = re.sub(CLEANR, '', data)
    return cleantext

In [11]:
strip_html(X_train.iloc[1])

'My wife and I both thought this film a watered-down, made-for-TV (BBC) version of Manhattan Murder Mystery...which is itself good, but not great. The story has little inter- character tension or chemistry, and not much of a plot. Woody Allen\'s character just sort of wanders around running off at the mouth, and Hugh Jackman and Scarlett Johannsson don\'t have a lot more to do. It\'s pretty disappointing, I must say. Ian McShane\'s role is just an expanded cameo appearance. The first thing that occurred to me was "I wonder how much the BBC had to pay Woody Allen to dislodge him from Manhatttan?" He must\'ve needed the money, and they must have needed his appeal to expand their audience beyond the youth market drawn to the two stars. I\'m giving this movie 4 stars instead of 3 because it is unbothersome background noise. If you ever want something to have on while you\'re knitting or sorting your stamp collection, this\'ll do the job. I wouldn\'t pay to rent it again.'

In [12]:
X_train= X_train.apply(strip_html)

# Tokanization

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [14]:
tokenizer= Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train)
X_train_seq= tokenizer.texts_to_sequences(X_train)


In [15]:
len(tokenizer.index_word)

114002

In [16]:
tokenizer.index_word[999]

'political'

In [17]:
len(X_train_seq[0]), len(X_train_seq[1]), len(X_train_seq[2])

(205, 166, 139)

In [18]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [19]:
X_train_seq_pad= pad_sequences(X_train_seq,maxlen=150)

In [20]:
len(X_train_seq_pad[0]), len(X_train_seq_pad[1]), len(X_train_seq_pad[2])

(150, 150, 150)

In [21]:
# Padding Done - Each Review is of len 150 now

In [22]:
X_train_seq_pad[2]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,   794,  1144,    44,     3,    51,   737,    64,
           5,    45,  2044,   178,   103,    59,  1504,  2309,    19,
           1,    64,    45,     4,    10,  2044,   178,   103,    22,
          26,  1064,   656, 11725,     2,  7951,    33,     1,   321,
           4,   794,  1144,     6,    51,  2477,     2,   570,    17,
           1,  1063,   508,     8,   115,    20,   117,    85,     5,
         273,   136,   168,    37,   218,     5,     1,  1724,     7,
         794,  1144,    10,    18,     6,   458,   599,    17,     3,
          51,   187,   599,    29,     1,   796,     4,   794,  1144,
        4031,     5,     1,   244,   702,     4,   103,  2094,   101,
         160,     4,   735,   352,    59,   161,   425,     2,   644,
         582,    43,   749,     5,   871,    11,     1,     6,  6703,
           5,  1027,   136,   212,    37,  1213,     9,    77,    20,
        2922,  1781,

In [23]:
y_train # As y_train is string .. need to convert it into num

4057     positive
20382    negative
4386     negative
42507    positive
11197    negative
           ...   
42697    positive
36008    positive
46265    positive
23587    positive
29313    negative
Name: sentiment, Length: 40000, dtype: object

In [24]:
y_train= y_train.apply(lambda X:1 if X=='positive' else 0)

In [25]:
y_train

4057     1
20382    0
4386     0
42507    1
11197    0
        ..
42697    1
36008    1
46265    1
23587    1
29313    0
Name: sentiment, Length: 40000, dtype: int64

In [26]:
#from keras.utils import to_categorical

In [27]:
#y_train = to_categorical(y_train)

# Model Building -LSTM

In [28]:
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.models import Sequential

In [36]:
model_1= Sequential()
model_1.add(LSTM(99, return_sequences=True))
model_1.add(LSTM(99, return_sequences=True))
model_1.add(LSTM(99, return_sequences=False))

model_1.add(Dense(1, activation='sigmoid'))

model_1.compile(loss='binary_crossentropy', metrics=['accuracy'])

In [37]:
# Train The Moddle On Google Colab

In [29]:
from tensorflow.keras.models import load_model

In [30]:
model_imdb= load_model('IMDB_Midel_1.h5')

In [31]:
# Preprocessing of Test data

In [32]:
# Remove HTML tags
X_test= X_test.apply(strip_html)

In [33]:
# Tokanization
X_test_seq= tokenizer.texts_to_sequences(X_test)


In [34]:
# Padding
X_test_seq_pad= pad_sequences(X_test_seq,maxlen=150)

In [38]:
y_test= y_test.apply(lambda X:1 if X=='positive' else 0)

In [39]:
y_test

8800     1
24061    0
1144     1
2703     0
5542     0
        ..
47944    0
5874     1
40426    0
41564    0
46091    0
Name: sentiment, Length: 10000, dtype: int64

In [41]:
model_imdb.evaluate(X_test_seq_pad,y_test,batch_size=32)

313/313 [==============================] - 75s 239ms/step - loss: 0.6667 - accuracy: 0.8614


[0.666710376739502, 0.8614000082015991]